## Import

- [ ] todo: change get_time to be a utility function in pgklUtil
- [ ] todo: consistent code units/physical units convention
- [ ] todo: plasmon density properly normalized
- [ ] todo: create separate notebook for processing pickled ts files
- [ ] todo: create separate notebook to run the functions in this module
- [ ] todo: use nbdev for all notebooks

In [1]:

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#from pylab import *
import postgkyl as pg
import numpy as np
import re
import os

import adios as ad
# import sys

# from shutil import copyfile
import pgkylUtil as pgu

# import subprocess
#import shlex


In [2]:
!ls ../data/ | head -15

Biskamp_newNoise_FixedJ_Tratio100.lua
Biskamp_newNoise_FixedJ_Tratio100_elc_0.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_1.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_2.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_3.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_4.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_5.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_6.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_7.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_8.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_9.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_0.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_1.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_2.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_3.bp


## Hardcode location of data files and directory for diagnostic output

In [2]:
#sys.path.insert(0, '/Users/rlw/Dropbox/ForBB/LucioFiles/pgkyl_script/')  

DATA_DIR = '/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/data/'
INPUT_FILENAME   = 'Biskamp_newNoise_FixedJ_Tratio100.lua'   #.Root name of files to process. #todo(Ryan): fix hardcode
FILEROOT = INPUT_FILENAME.split('.lua', maxsplit=1)[0]

OUTPUT_DIR     = '/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/'               #.Output directory where figures are saved.

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [4]:
#!rm /Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/*

## `get_max_filenumber`

Making sure my regular expression works

In [3]:
test_file_prefix = "Biskamp_newNoise_FixedJ_Tratio100"
test_filename = "Biskamp_newNoise_FixedJ_Tratio100_elc_2.bp"

el = re.compile(rf'{test_file_prefix}_elc_\d+.bp')
el.search(test_filename)

<re.Match object; span=(0, 42), match='Biskamp_newNoise_FixedJ_Tratio100_elc_2.bp'>

In [4]:
#export
def get_max_filenumber(data_dir: str, input_filename: str) -> int:
    """Return the filenumber of the largest numbered output file in <data_dir> generated by <input_filename>
    
    data_dir: path to directory containing .bp files
    input_filename: e.g. 'Biskamp_2x2v.lua'
    """

    file_prefix = input_filename.split('.lua', maxsplit=1)[0]
    
    elc_pattern = re.compile(rf'{file_prefix}_elc_\d+.bp')
    elc_output_filenames = [filename for filename in os.listdir(data_dir) if elc_pattern.search(filename)]
    
    filenumbers = []
    for filename in elc_output_filenames:
            filenumber = re.findall(r'\d+.bp', filename)[0].split('.bp',1)[0]
            filenumbers.append(filenumber)
    
    max_filenumber = int(max(filenumbers))
    return max_filenumber

In [5]:
test_max_filenumber = get_max_filenumber(DATA_DIR, INPUT_FILENAME)
test_max_filenumber

9

# The full `main_plotting` function

## Encapsulated functions for getting data

### `get_grid_data`

In [6]:
def get_grid_data(fname_prefix, nFr, data_dir='../data/', poly_order=2, basis_type='ms'):
    #.Extract grid details from one of the data files for each species
    fName_elc = data_dir + fname_prefix+'_elc_'+str(nFr)+'.bp'
    fName_ion = data_dir + fname_prefix+'_ion_'+str(nFr)+'.bp'

    # getGrid data
    x_elc, _, nx, lx, _ = pgu.getGrid(fName_elc, poly_order, basis_type, location='center')
    x_ion, _, _, _, _ = pgu.getGrid(fName_ion, poly_order, basis_type, location='center')

    # Unpack needed data from getGrid
    
    n_z = nx[0]  # number of gridpoints along z
    n_y = nx[1]  # number of gridpoints along y

    lz = lx[0]  #box length along z
    ly = lx[1]  #box length along y

    # Setting up the grids in X space for plotting
    points_z = np.array(x_elc[0])
    points_y = np.array(x_elc[1])   
    grid_z, grid_y = np.meshgrid(points_z, points_y,indexing='ij')
    
    # Setting up the grids in V space for plotting
    #    electrons
    velocitiesz_elc = np.array(x_elc[2]) 
    velocitiesy_elc = np.array(x_elc[3]) 
    Vz_elc, Vy_elc = np.meshgrid(velocitiesz_elc,velocitiesy_elc,indexing='ij')    
    #    ions
    velocitiesz_ion = np.array(x_ion[2])
    velocitiesy_ion = np.array(x_ion[3])
    Vz_ion, Vy_ion = np.meshgrid(velocitiesz_ion,velocitiesy_ion,indexing='ij')

    # Setting up the grids in k space for plotting
    kz_plot   = 2.0*3.14159*np.linspace(-int(n_z/2), int(n_z/2-1), n_z)/lz
    ky_plot  = 2.0*3.14159*np.linspace(-int(n_y/2), int(n_y/2-1), n_y)/ly
    K_z, K_y = np.meshgrid(kz_plot, ky_plot, indexing = 'xy')
    
    return (Vz_elc, Vy_elc, Vz_ion, Vy_ion, grid_z, grid_y, n_z, n_y, K_z, K_y)

In [7]:
test_fname_prefix = "Biskamp_newNoise_FixedJ_Tratio100"
test_nFr = 0
test_grid_data = get_grid_data(test_fname_prefix, test_nFr, data_dir=DATA_DIR, poly_order=2, basis_type='ms')

/Users/rlw/anaconda3/envs/pgkyl-dev/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [8]:
print("n_z =", test_grid_data[6])
print("n_y =", test_grid_data[7])

n_z = 96
n_y = 48


### `get_species_data`

In [9]:
def get_species_data(fname_prefix, nFr, species_code, mass=1.0, data_dir='../data/',  poly_order=2, basis_type='ms'):
    """Return interpolated distribution function data (in code units) for chosen species at chosen frame number
    """

    #     distribution function
    filename_f = data_dir + fname_prefix+'_'+ species_code + '_'+str(nFr)+'.bp'
    
    #     moments
    filename_M0 = data_dir + fname_prefix + '_' +  species_code + '_M0_'+str(nFr)+'.bp'
    filename_M1 = data_dir + fname_prefix+ '_' +  species_code + '_M1i_'+str(nFr)+'.bp'        
    filename_vT = data_dir + fname_prefix+ '_' +  species_code + '_vthSq_'+str(nFr)+'.bp'
    filename_u = data_dir + fname_prefix+ '_' +  species_code + '_u_'+str(nFr)+'.bp'

    
    f = np.squeeze(pgu.getInterpData(filename_f, poly_order, basis_type))
    n = np.squeeze(pgu.getInterpData(filename_M0, poly_order, basis_type, comp=0))
    nu_z = np.squeeze(pgu.getInterpData(filename_M1, poly_order, basis_type, comp=0))
    nu_y = np.squeeze(pgu.getInterpData(filename_M1, poly_order,basis_type, comp=1))
    u_z = np.squeeze(pgu.getInterpData(filename_u, poly_order,basis_type, comp=0))
    u_y = np.squeeze(pgu.getInterpData(filename_u, poly_order,basis_type, comp=1))
    vTsq = np.squeeze(pgu.getInterpData(filename_vT, poly_order, basis_type))
    
    vT = np.sqrt(vTsq)
    T = mass * vTsq  #This is an array of temperature values: one for each spatial location
    T_boxavg = np.average(T)
    return (f, n, nu_z, nu_y, u_z, u_y, vTsq, vT, T, T_boxavg)

In [10]:
test_fname_prefix = "Biskamp_newNoise_FixedJ_Tratio100"
test_nFr = 0
test_species_data = get_species_data(test_fname_prefix, test_nFr, 'ion', mass=25.0, data_dir=DATA_DIR, poly_order=2, basis_type='ms')

### `get_field_data`

In [11]:
def get_field_data(fname_prefix, nFr, data_dir='../data/', poly_order=2, basis_type='ms'):
    """Return interpolated field data (in code units) for the given frame number"""
    fName_field = data_dir + fname_prefix+'_'+ 'field' + '_'+str(nFr)+'.bp'
    field_E_z = np.squeeze(pgu.getInterpData(fName_field,poly_order,basis_type,comp=0)) # JJ: can this be turned into 1 call without comp specified?
    field_E_y = np.squeeze(pgu.getInterpData(fName_field,poly_order,basis_type,comp=1))
    
    return (field_E_z, field_E_y)

In [12]:
test_fname_prefix = "Biskamp_newNoise_FixedJ_Tratio100"
test_nFr = 0
test_species_data = get_field_data(test_fname_prefix, test_nFr, data_dir=DATA_DIR, poly_order=2, basis_type='ms')

### `get_time`

In [13]:
def get_time(fname_prefix, nFr, data_dir='../data/'):
    """Return time (in code units) """
                #.Extract the time from file.
    with ad.file(data_dir + fname_prefix + '_elc_' + str(nFr) + '.bp') as hF:
        time = hF['time'].read()
    return time

In [14]:
test_fname_prefix = "Biskamp_newNoise_FixedJ_Tratio100"
test_nFr = 7
get_time(test_fname_prefix, test_nFr, data_dir=DATA_DIR)

<ipython-input-13-c7dedd19b8fc>:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  time = hF['time'].read()


0.44156249999999947

### `create_dist_plots_file`

In [31]:
def create_dist_plots_file(nFr, elc_species_data, ion_species_data, field_data):
    
    elc_f, _, elc_M1_z, elc_M1_y, elc_u_z, elc_u_y, elc_vTsq, elc_vT, elc_T, elc_T_boxavg = elc_species_data
    ion_f, n_ion, ion_M1_z, ion_M1_y, ion_u_z, ion_u_y, ion_vTsq, ion_vT, _, ion_T_boxavg = ion_species_data
    field_E_z, field_E_y = field_data

    
    # get distribution function values i) averaged over space, ii) at a fixed location in space
    elc_f_boxavg = np.average(elc_f, axis = (0,1)) #average over both spatial dimensions
    ion_f_boxavg = np.average(ion_f, axis = (0,1))
    
    elc_f_local = elc_f[int(elc_f.shape[0]/2), int(elc_f.shape[1]/2), :, :]
    ion_f_local = ion_f[int(ion_f.shape[0]/2), int(ion_f.shape[1]/2), :, :]
    
    
    elc_u_z_boxavg = np.average(elc_u_z)
    elc_u_y_boxavg = np.average(elc_u_y)
    
    # Compute current density.
    J_z = ion_M1_z - elc_M1_z
    J_y = ion_M1_y - elc_M1_y
    # n_ion = ionM0

    J_fluct_z_k = np.fft.fftn(J_z - np.average(J_z))
    J_fluct_y_k = np.fft.fftn(J_y - np.average(J_y))
    field_E_fluct_z_k = np.fft.fftn(field_E_z - np.average(field_E_z)) 
    field_E_fluct_y_k = np.fft.fftn(field_E_y - np.average(field_E_y))
    
    JdotE_k = np.abs(np.transpose(np.fft.fftshift(J_fluct_z_k*field_E_fluct_z_k + J_fluct_y_k*field_E_fluct_y_k) ) )
    field_E_fluct_square_K = np.abs(np.transpose(np.fft.fftshift(field_E_fluct_z_k**2 + field_E_fluct_y_k**2) ) )      
    
    
    
    time = get_time(FILEROOT, nFr)
    t_formatted = float('%.3g' % time)  

    fignum = str(nFr).zfill(4)

    #Distribution function plots: local and box average
    fig, axs = plt.subplots(2,3,figsize=(40, 20), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace =.1)
    axs = axs.ravel()

    pos0 = axs[0].pcolormesh(elc_v_z/c_sound_0, elc_v_y/c_sound_0, elc_f_local)
    axs[0].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[0].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[0].set_title(r'$F_e(z_0,y_0,v_z,v_y),$', fontsize=26)
    axs[0].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos0, ax=axs[0])
    cbar.ax.tick_params(labelsize=22)

    pos1 = axs[1].pcolormesh(ion_v_z/c_sound_0, ion_v_y/c_sound_0, ion_f_local)
    axs[1].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[1].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[1].set_title(r'$F_i(z_0,y_0,v_z,v_y),$' + rf't = {t_formatted}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[1].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos1, ax=axs[1])
    cbar.ax.tick_params(labelsize=22)

    pos2 = axs[2].pcolormesh(grid_z, grid_y, n_ion)
    axs[2].set_xlabel(r'$z \ [d_e]$', fontsize=30)
    axs[2].set_ylabel(r'$y \ [d_e]$', fontsize=30, labelpad=-1)
    axs[2].set_title(r'$n_i(z,y)$', fontsize=30)
    axs[2].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos2, ax=axs[2])
    cbar.ax.tick_params(labelsize=22)
#     pos2 = axs[2].pcolormesh(grid_y, grid_z, J_z)
#     axs[2].set_xlabel(r'$y \ [d_e]$', fontsize=30)
#     axs[2].set_ylabel(r'$z \ [d_e]$', fontsize=30, labelpad=-1)
#     axs[2].set_title(r'$J_z(z,y)$', fontsize=30)
#     axs[2].tick_params(labelsize = 26)
#     cbar = fig.colorbar(pos2, ax=axs[2])
#     cbar.ax.tick_params(labelsize=22)

    pos3 = axs[3].pcolormesh(elc_v_z/c_sound_0, elc_v_y/c_sound_0, elc_f_boxavg)
    axs[3].scatter(elc_u_z_boxavg, elc_u_y_boxavg, s = 60)
    axs[3].scatter(np.squeeze(elc_v_z[np.where(elc_f_boxavg==np.max(elc_f_boxavg))]),np.squeeze(elc_v_y[np.where(elc_f_boxavg==np.max(elc_f_boxavg))]),s = 40, marker = 'x', alpha = 1)
    axs[3].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[3].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[3].set_title(r'$<F_e(v_z,v_y)>_{z,y},$' + rf't = {t_formatted}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[3].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos3, ax=axs[3])
    cbar.ax.tick_params(labelsize=22)

    pos4 = axs[4].pcolormesh(ion_v_z/c_sound_0, ion_v_y/c_sound_0, ion_f_boxavg)
    axs[4].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[4].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[4].set_title(r'$<F_i(v_z,v_y)>_{z,y},$' + rf't = {t_formatted}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[4].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos4, ax=axs[4])
    cbar.ax.tick_params(labelsize=22)

    pos5 = axs[5].contourf(V_TE0*K_z, V_TE0*K_y, field_E_fluct_square_K)
    axs[5].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
    axs[5].set_ylabel(r'$k_y \lambda_{De0}$', fontsize=30, labelpad=-1)
    axs[5].set_title(rf'$|\delta E^2|_k\propto N(k_x, k_y)$, t = {t_formatted}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
    axs[5].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos5, ax=axs[5])
    cbar.ax.tick_params(labelsize=22) 

    


#     pos5 = axs[5].contourf(V_TE0*K_z, V_TE0*K_y, JdotE_k)
#     axs[5].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
#     axs[5].set_ylabel(r'$k_y \lambda_{De0}$', fontsize=30, labelpad=-1)
#     axs[5].set_title(rf'$(\delta J \cdot \delta E)_k$, t = {t_formatted}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
#     axs[5].tick_params(labelsize = 26)
#     cbar = fig.colorbar(pos5, ax=axs[5])
#     cbar.ax.tick_params(labelsize=22)

    fig.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, FILEROOT) + rf'_distributions_{fignum}.png', bbox_inches='tight')
    plt.close()


## `save_ts_data`

In [67]:
def save_ts_data(ts_var2file_map):
    for ts_var, path in ts_var2file_map:
        np.save(path, ts_var)
    return

In [91]:
# setup
test_t__ts_filepath = os.path.join(OUTPUT_DIR, FILEROOT+rf'_test_t__ts.npy')
test_f_frame = 2
test_t__ts = np.ones(test_f_frame)

test_ts_var2file_map = [
    (test_t__ts, test_t__ts_filepath)
]

# test
save_ts_data(test_ts_var2file_map)
np.testing.assert_array_equal(np.load(test_t__ts_filepath),np.array([1.0, 1.0]))

# tear down
os.remove(test_t__ts_filepath)
del test_t__ts_filepath, test_f_frame, test_t__ts, test_ts_var2file_map

In [90]:
os.path.exists(os.path.join(OUTPUT_DIR, FILEROOT+rf'_test_t__ts.npy'))

False

In [75]:
test_t__ts_filepath

'/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/Biskamp_newNoise_FixedJ_Tratio100_test_t__ts.npy'

In [76]:
ls /Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/Biskamp_newNoise_FixedJ_Tratio100_test_t__ts.npy

/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/Biskamp_newNoise_FixedJ_Tratio100_test_t__ts.npy


In [77]:
np.load(test_t__ts_filepath)

array([1., 1.])

## `load_saved_ts_data`

In [16]:
def load_saved_ts_data(ts_var2file_map, f_frame):

    for ts_var, path in ts_var2file_map:
        if os.path.exists(path):
            saved_ts = np.load(path)
            saved_f_frame = len(saved_ts)
            ts_var[:saved_f_frame] = saved_ts.copy()
    return

In [78]:
test_f_frame = 4
test_t__ts = np.zeros(test_f_frame)

test_ts_var2file_map = [
    (test_t__ts, test_t__ts_filepath)
]



print(test_t__ts)
print(test_old_t__ts)
load_saved_ts_data(test_ts_var2file_map, test_f_frame)
print(test_t__ts)

[0. 0. 0. 0.]
[1. 1.]
[1. 1. 0. 0.]


In [79]:
del test_f_frame, test_t__ts, test_ts_var2file_map 

## `get_i_frame`

In [19]:
def get_i_frame(output_dir, fileroot):
    
    i_frame = 0  # starting assumption
    
    if os.path.exists(output_dir):
        output_files = os.listdir(output_dir)
        # get list of frame numbers for all of the png files in `output_dir`
        nFrs = [int(re.findall(r"\d{4}.png", file)[0][:4]) for file in output_files if len(re.findall(r"\d{4}.png", file)) == 1]
        if len(nFrs) > 0:
            max_nFr_pngs = max(nFrs)
            i_frame = max_nFr_pngs + 1
#         if os.path.exists(os.path.join(output_dir, os.path.join(OUTPUT_DIR, FILEROOT)+rf'_t__ts')):
#             t__ts = np.load(os.path.join(output_dir, os.path.join(OUTPUT_DIR, FILEROOT)+rf'_t__ts')):
#             max_nFr__ts = len(t__ts)
    
    return i_frame
    #t__ts = np.load(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_t__ts')
    
    #return len(t__ts) + 1

In [24]:
get_i_frame('/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/', FILEROOT)

10

## Computations

In [17]:
def get_local_f(f: np.array):
    z0 = int(f.shape[0]/2)
    y0 = int(f.shape[1]/2)
    
    local_f = f[z0, y0, :, :]
    return local_f

In [18]:
test_fname_prefix = "Biskamp_newNoise_FixedJ_Tratio100"
test_nFr = 0
test_species_data = get_species_data(test_fname_prefix, test_nFr, 'elc', mass=1.0, data_dir=DATA_DIR, poly_order=2, basis_type='ms')
test_f_elc = test_species_data[0] 

test_local_f_elc = get_local_f(test_f_elc)

# plt.pcolormesh(test_local_f_elc)
# plt.show()

## Extract simulation data from lua file

In [16]:
test_luafile = "Biskamp_newNoise_FixedJ_Tratio100.lua"

In [19]:
filepath = os.path.join(DATA_DIR, test_luafile)
with open(filepath) as fh:
    dat = fh.read()
    
dat.split('\n')[:30]

['-- Gkyl ------------------------------------------------------------------------',
 '--Notes',
 '-- 0. The following modifications have been made ',
 '--      0.1 Get diagnostic temperature output for electrons ',
 '--      0.2 Unfortunately, this input file was a modification of an input file that had a "bad" noise function.  The waves have the form sin(pi x) rather than sin(2 pi x), so there is a wave whose period is too large, probably causing boundary issues.',
 '-- 1. Biskamp had a total run time of t = 2000 omega_pe^-1.  His plots show that the fluctuation energy increases until t = 800-1000, and then there is an ion heating phase, which is also a runaway production phase.',
 '--      1.1. Biskamp used a mass ratio of 400 for the plots shown.  The meaningful time is omega_pi, so t = 2000 (Sqrt[m_e/m_i]Biskamp) omega_pe^-1 = 2000*Sqrt(25/400) omega_pe^-1 = 2000/4 omega_pe^-1 = 500 omega_pe^-1. ',
 '--      1.2. We can run the sim for 1/2 the time, make plots, then run for the re

```
basis       = "serendipity",    -- One of "serendipity" or "maximal-order".',
polyOrder   = pOrder,
```

In [20]:
def get_luafile_data(luafile):
    filepath = os.path.join(DATA_DIR, test_luafile)
    with open(filepath) as fh:
        fh.read()

## Load in pickled time series data from previous diagnostic runs

In [17]:
OUTPUT_DIR + FILEROOT

'/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/Biskamp_newNoise_FixedJ_Tratio100'

In [26]:
os.path.join(OUTPUT_DIR, FILEROOT)

'/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/Biskamp_newNoise_FixedJ_Tratio100'

In [18]:
!ls 

01_refactor_main_plotting.ipynb gkyl_plots_v8
02_refactor_pgkylUtil.ipynb     gkyl_plots_v8_refactor
__pycache__                     pgkylUtil.py


In [19]:
def init_ts_arrays():
    """
    """
    np.zeros()
    if <previously_saved_array_exists>:
        np.load()
    else:
        

SyntaxError: invalid syntax (<ipython-input-19-da6b5adb5d8d>, line 5)

In [ ]:
def save_ts_arrays():

In [ ]:
# t__ts = np.zeros(nFrames)
# E_z_boxavg__ts = np.zeros(nFrames)
# J_z_boxavg__ts = np.zeros(nFrames)
# E_fluct_squared_boxavg__ts   = np.zeros(nFrames)
# ion_T_boxavg__ts = np.zeros(nFrames)
# elc_T_boxavg__ts = np.zeros(nFrames)

In [16]:
np.load(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_timesFile'+'.npy')

NameError: name 'FILEROOT' is not defined

In [22]:
!ls ../gkyl_plots_v8_refactor

ls: ../gkyl_plots_v8_refactor: No such file or directory


# Run remaining `main_plotting` function

In [21]:
os.chdir(DATA_DIR)

In [22]:
MAX_FILENUMBER = get_max_filenumber(DATA_DIR, INPUT_FILENAME)

handjam the data from the input file

In [25]:
# numerical
POLYORDER  = 2                   
BASISTYPE  = 'ms'              

# physical
M_ION = 25                   
V_TE0 = 0.02                 
ALPHA = 0.00              

c_sound_0 = V_TE0/np.sqrt(M_ION) 

#FOURIER_TRANSFORM = True


In [90]:
!ls '/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/'

Biskamp_newNoise_FixedJ_Tratio100_distributions_0000.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0001.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0002.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0003.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0004.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0005.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0006.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0007.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0008.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0009.png


In [92]:
os.path.exists("/Users/rlw/gh_repos/Gkeyll_IAT/Diagnostics/2x2v/nbs/gkyl_plots_v8_refactor/Biskamp_newNoise_FixedJ_Tratio100_distributions_0000.png")

True

In [26]:
iFrame     = 0     #.Initial frame (0 is the t=0 frame).
fFrame     = MAX_FILENUMBER    #.Final frame.
nFrames = fFrame-iFrame+1    #.Number of frames plotted.


elc_v_z, elc_v_y, ion_v_z, ion_v_y, grid_z, grid_y, n_z, n_y, K_z, K_y = get_grid_data(FILEROOT, iFrame, data_dir=DATA_DIR, poly_order=POLYORDER, basis_type=BASISTYPE)

In [40]:
# initialize time series variables
t__ts = np.zeros(nFrames)
E_z_boxavg__ts = np.zeros(nFrames)
J_z_boxavg__ts = np.zeros(nFrames)
E_fluct_squared_boxavg__ts = np.zeros(nFrames)
ion_T_boxavg__ts = np.zeros(nFrames)
elc_T_boxavg__ts = np.zeros(nFrames)

ts_arrays = (t__ts, 
    J_z_boxavg__ts, 
    E_z_boxavg__ts, 
    ion_T_boxavg__ts, 
    elc_T_boxavg__ts, 
    E_fluct_squared_boxavg__ts
)

ts_var2file_map = [
    (t__ts, os.path.join(OUTPUT_DIR, FILEROOT, rf'_t__ts.npy')),
    (E_z_boxavg__ts, os.path.join(OUTPUT_DIR, FILEROOT, rf'_E_z_boxavg__ts.npy')),
    (J_z_boxavg__ts, os.path.join(OUTPUT_DIR, FILEROOT, rf'_E_z_boxavg__ts.npy')),
    (E_fluct_squared_boxavg__ts, os.path.join(OUTPUT_DIR, FILEROOT, rf'_E_z_boxavg__ts.npy')),
    (ion_T_boxavg__ts, os.path.join(OUTPUT_DIR, FILEROOT, rf'_E_z_boxavg__ts.npy')),
    (elc_T_boxavg__ts, os.path.join(OUTPUT_DIR, FILEROOT, rf'_ion_T_boxavg__ts.npy'))
]

load_saved_ts_data(ts_var2file_map, fFrame)

In [41]:
def compute_ts_quantities(nFr, elc_species_data, ion_species_data, field_data):
    # (f, n, nu_z, nu_y, u_z, u_y, vTsq, vT, T, T_boxavg)
    
    
    _, _, ion_M1_z, _, _, _, _, _, _, ion_T_boxavg = ion_species_data
    _, _, elc_M1_z, _, _, _, _, _, _, elc_T_boxavg = elc_species_data
    field_E_z, field_E_y = field_data
#     elc_u_z_boxavg = np.average(elc_u_z)
#     elc_u_y_boxavg = np.average(elc_u_y)
    
    
    # Compute current density.
    J_z = ion_M1_z - elc_M1_z
#    J_y = ion_M1_y - elc_M1_y
    # n_ion = ionM0


    t__nFr = get_time(FILEROOT, nFr)
    
    J_z_boxavg__nFr = np.average(J_z)/V_TE0
    E_z_boxavg__nFr = np.average(field_E_z)
    

    #compute energy 
    ion_T_boxavg__nFr = ion_T_boxavg/(V_TE0**2)
    elc_T_boxavg__nFr = elc_T_boxavg/(V_TE0**2)
    E_fluct_squared_boxavg__nFr = np.average((field_E_z - np.average(field_E_z))**2 + (field_E_y - np.average(field_E_y))**2)/V_TE0**2
    
    return (t__nFr, J_z_boxavg__nFr, E_z_boxavg__nFr, ion_T_boxavg__nFr, elc_T_boxavg__nFr, E_fluct_squared_boxavg__nFr)

In [42]:
test_nFr = 5
compute_ts_quantities(nFr, elc_species_data, ion_species_data, field_data)

<ipython-input-13-c7dedd19b8fc>:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  time = hF['time'].read()


(0.0,
 -4.029341207845271,
 -2e-05,
 0.018213196413376962,
 1.0000000822371364,
 2.5509860043886447e-38)

In [54]:
for nFr in range(iFrame,fFrame+1):
    
    elc_species_data = get_species_data(FILEROOT, nFr, 'elc', data_dir=DATA_DIR, poly_order=POLYORDER, basis_type=BASISTYPE)
    ion_species_data = get_species_data(FILEROOT, nFr, 'ion', mass=M_ION, data_dir=DATA_DIR, poly_order=POLYORDER, basis_type=BASISTYPE)
    field_data = get_field_data(FILEROOT, nFr, data_dir=DATA_DIR, poly_order=POLYORDER, basis_type=BASISTYPE)
    
    ts_quantities = compute_ts_quantities(nFr, elc_species_data, ion_species_data, field_data)
    
    for idx, ts_array in enumerate(ts_arrays):
        ts_array[nFr - iFrame] = ts_quantities[idx]
    
    
    time = get_time(FILEROOT, nFr)
    t_formatted = float('%.3g' % time)  

    fignum = str(nFr).zfill(4)
    create_dist_plots_file(nFr, elc_species_data, ion_species_data, field_data)

<ipython-input-13-c7dedd19b8fc>:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  time = hF['time'].read()
<ipython-input-31-cd84a9a07ff2>:44: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  pos0 = axs[0].pcolormesh(elc_v_z/c_sound_0, elc_v_y/c_sound_0, elc_f_local)
<ipython-input-31-cd84a9a07ff2>:52: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  pos1 = axs[1].pcolormesh(ion_v_z/c_sound_0, ion_v_y/c_sound_0, ion_f_lo

In [55]:
t__ts

array([0.                , 0.0646875         , 0.1265624999999999,
       0.1912499999999998, 0.2531249999999998, 0.3149999999999997,
       0.3796874999999996, 0.4415624999999995, 0.5034374999999994,
       0.5681249999999993])

## Compute time series dashboard

In [61]:
np.average(t__ts[1:4])

0.12749999999999995

In [65]:
def compute_rolling_avg(series: np.array, n_agg=2):
    """
    """
    rolling_series = series.copy()
    n_frames = len(rolling_series)
    
    for n in range(n_agg, n_frames-n_agg):
        print("n", n)
        rolling_series[n] = np.average(series[n-n_agg:n+n_agg])
    
    for n in range(n_agg):
        print("n_left", n)
        series[n] = series[n_agg]
        
    for n in range(n_frames-n_agg, n_frames):
        print("n_right", n)
        series[n] = series[n_frames-n_agg]
    
    return rolling_series

In [66]:
compute_rolling_avg(t__ts, 2)

n 2
n 3
n 4
n 5
n 6
n 7
n_left 0
n_left 1
n_right 8
n_right 9


array([0.                , 0.0646875         , 0.095625          ,
       0.1589062499999999, 0.2214843749999998, 0.2847656249999997,
       0.3473437499999996, 0.4099218749999995, 0.5034374999999994,
       0.5681249999999993])

In [ ]:
#e_fluct_squared_by_T__ts = np.zeros(nFrames)
#Rei = np.zeros(nFrames)
#e_fluct_squared_byT__ts = np.zeros(nFrames)
# E_over_J_rolling = np.zeros(nFrames)
# R_over_J_rolling = np.zeros(nFrames)

# E_boxavg_over_J_boxavg = np.zeros(nFrames)

# nu_Sagdeev = np.zeros(nFrames)

In [26]:
Navg = 3
R_ei__ts = E_z_boxavg__ts + ALPHA*V_TE0
#    E_over_J_rolling = np.zeros(nFrames)
for n in range(Navg,nFrames):
    E_over_J_rolling[n] = np.sum(E_z_boxavg__ts[n-Navg:n])/np.sum(J_z_boxavg__ts[n-Navg:n])
for n in range(Navg):
    E_over_J_rolling[n] =  E_over_J_rolling[Navg]  #bfill the first Navg values


for n in range(Navg,nFrames):
    R_over_J_rolling[n] = np.sum(R_ei__ts[n-Navg:n])/np.sum(J_z_boxavg__ts[n-Navg:n])
for n in range(Navg):
    R_over_J_rolling[n] =  R_over_J_rolling[Navg]  #bfill the first Navg values
nu_eff_rolling = np.absolute(R_over_J_rolling)

nu_Sagdeev = 0.025*np.absolute(J_z_boxavg__ts/np.sqrt(elc_T_boxavg__ts))*elc_T_boxavg__ts/ion_T_boxavg__ts

In [26]:
np.save(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_t__ts',t__ts[0:fFrame-iFrame+1])
np.save(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_E_z_boxavg__ts',E_z_boxavg__ts[0:fFrame-iFrame+1])
np.save(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_J_z_boxavg__ts',J_z_boxavg__ts[0:fFrame-iFrame+1])
np.save(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_E_fluct_squared_boxavg__ts',E_fluct_squared_boxavg__ts[0:fFrame-iFrame+1])
np.save(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_ion_T_boxavg__ts',ion_T_boxavg__ts[0:fFrame-iFrame+1])
np.save(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_ion_T_boxavg__ts',elc_T_boxavg__ts[0:fFrame-iFrame+1])
#np.save(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_R_ei_boxavgFile',elc_T_boxavg__ts[0:fFrame-iFrame+1])

In [26]:
##########################
#Energy plots
fig, axs = plt.subplots(2,3,figsize=(30, 20), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace =.1)
axs = axs.ravel()



axs[0].plot(t__ts[0:fFrame-iFrame+1],E_fluct_squared_boxavg__ts,label=r'$ \left(\epsilon_0 \langle|\delta {E}|^2\rangle_{x,y}/2\right)/ T_{e0} $')
axs[0].plot(t__ts[0:fFrame-iFrame+1],E_fluct_squared_boxavg__ts/elc_T_boxavg__ts,label=r'$ \left(\epsilon_0 \langle|\delta {E}|^2\rangle_{x,y}/2\right)/ \langle T_{e} \rangle $')
axs[0].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[0].set_ylabel(r'$ \left(\epsilon_0 \langle|\delta {E}|^2\rangle_{x,y}/2\right)/ T_{e0} $', fontsize=30)
axs[0].tick_params(labelsize = 26)
axs[0].legend(fontsize = 18)

axs[1].plot(t__ts[0:fFrame-iFrame+1],E_z_boxavg__ts)
axs[1].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[1].set_ylabel(r'$\langle E_z \rangle$', fontsize=30)
axs[1].tick_params(labelsize = 26)
#    axs[1].set_ylim(top = 0.00)

axs[2].plot(t__ts[0:fFrame-iFrame+1],E_over_J_rolling)
axs[2].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[2].set_ylabel(r'$\langle E_z\rangle /\langle\, J_z\rangle \ [\nu_{\mathrm{eff}}/ \omega_{pe}]$', fontsize=30)
axs[2].tick_params(labelsize = 26)
#    axs[2].set_ylim(0.0, 2*np.amax(E_over_J_rolling))

axs[3].plot(t__ts[0:fFrame-iFrame+1],elc_T_boxavg__ts)
axs[3].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[3].set_ylabel(r'$T_e /T_{e0}$', fontsize=30)
axs[3].tick_params(labelsize = 26)
axs[3].set_ylim(0.0, 2*np.amax(elc_T_boxavg__ts) )

axs[4].plot(t__ts[0:fFrame-iFrame+1],ion_T_boxavg__ts, label = r'$T_i /T_{e0}$')
axs[4].plot(t__ts[0:fFrame-iFrame+1],ion_T_boxavg__ts/elc_T_boxavg__ts, label = r'$T_i /T_{e}$')
axs[4].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[4].set_ylabel(r'$T_i /T_{e0}$', fontsize=30)
axs[4].tick_params(labelsize = 26)
axs[4].set_ylim(0.0, 0.1)
axs[4].legend(fontsize = 18)
#    axs[4].set_ylim(0.0, 2*np.amax(ion_T_boxavg__ts) )

axs[5].plot(t__ts[0:fFrame-iFrame+1],nu_eff_rolling, label = 'nu_eff')
axs[5].plot(t__ts[0:fFrame-iFrame+1],nu_Sagdeev/10.0, label = 'nu_Sagdeev / 10')
axs[5].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[5].set_ylabel(r'$\langle R_z\rangle /\langle\, J_z\rangle \ [\nu_{\mathrm{eff}}/ \omega_{pe}]$', fontsize=30)
axs[5].tick_params(labelsize = 26)
axs[5].legend(fontsize = 18)

fig.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_reconnection_plots.png', bbox_inches='tight')
plt.close()

In [23]:
#!ls gkyl_plots_v8_refactor/Biskamp_newNoise_FixedJ_Tratio100_distributions_0000.png

In [24]:
#!ls 

# Extra junk

In [ ]:
def create_fourier_plots():
    
#     # split into spatial average and fluctuating parts

    

    #e_fluct_squared_by_T__ts[nFr-iFrame] = E_fluct_squared_boxavg__ts[nFr-iFrame] / elc_T_boxavg__ts[nFr-iFrame]




    
    
    #################################################################################################################

    
#     field_E_fluct_z_k = np.fft.fftn(field_E_fluct_z) #note: I think that the division be VT0 is essentially a Debye length normalization
#     field_E_fluct_y_k = np.fft.fftn(field_E_fluct_y)

    #print('fft_freq', np.fft.fftfreq(128) )
#     J_fluct_z = J_z - np.average(J_z)
#     J_fluct_y = J_y - np.average(J_y)
#     J_fluct_z_k = np.fft.fftn(J_fluct_z)
#     J_fluct_y_k = np.fft.fftn(J_fluct_y)
    
    J_fluct_z_k = np.fft.fftn(J_z - np.average(J_z))
    J_fluct_y_k = np.fft.fftn(J_y - np.average(J_y))
    

#     field_E_z_boxavg = np.average(field_E_z)
#     field_E_y_boxavg = np.average(field_E_y)
#     field_E_fluct_z = field_E_z - field_E_z_boxavg
#     field_E_fluct_y = field_E_y - field_E_y_boxavg    
    field_E_fluct_z_k = np.fft.fftn(field_E_z - np.average(field_E_z)) 
    field_E_fluct_y_k = np.fft.fftn(field_E_z - np.average(field_E_z))
    

    JdotE_k = np.abs(np.transpose(np.fft.fftshift(J_fluct_z_k*field_E_fluct_z_k + J_fluct_y_k*field_E_fluct_y_k) ) )
    field_E_fluct_square_K = np.abs(np.transpose(np.fft.fftshift(field_E_fluct_z_k**2 + field_E_fluct_y_k**2) ) )  
    
    
    fignum = str(nFr).zfill(4)
    fig, axs = plt.subplots(2,2,figsize=(20, 20), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace =.1)
    axs = axs.ravel()
    pos0 = axs[0].pcolormesh(elc_v_z, elc_v_y, elc_f_boxavg)
    axs[0].scatter(elc_u_z_boxavg, elc_u_y_boxavg, s = 60)
    axs[0].scatter(np.squeeze(elc_v_z[np.where(elc_f_boxavg==np.max(elc_f_boxavg))]),np.squeeze(elc_v_y[np.where(elc_f_boxavg==np.max(elc_f_boxavg))]),s = 40, marker = 'x', alpha = 1)
    axs[0].set_xlabel(r'$v_z/c$', fontsize=30)
    axs[0].set_ylabel(r'$v_y/c$', fontsize=30, labelpad=-1)
    axs[0].set_title(r'$<F_e(v_z,v_y)>_{z,y},$' + rf't = {t_formatted}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[0].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos0, ax=axs[0])
    cbar.ax.tick_params(labelsize=22)

    pos1 = axs[1].pcolormesh(ion_v_z, ion_v_y, ion_f_boxavg)
    axs[1].set_xlabel(r'$v_z/c$', fontsize=30)
    axs[1].set_ylabel(r'$v_y/c$', fontsize=30, labelpad=-1)
    axs[1].set_title(r'$<F_i(v_z,v_y)>_{z,y},$' + rf't = {t_formatted}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[1].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos1, ax=axs[1])
    cbar.ax.tick_params(labelsize=22)
#            pos0 = axs[0].plot(e_z_raw_cell_average_k_y_int)
#            axs[0].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
#            axs[0].set_ylabel(r'$E(k_z)$', fontsize=30, labelpad=-1)
#            axs[0].set_title(rf't = {time}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
#            axs[0].tick_params(labelsize = 26)
#            axs[0].set_yscale('symlog') #plot on log scale and take absolute value of number

#            pos1 = axs[1].plot(J_z_k_raw_y_int)
#            axs[1].set_xlabel(r'$k_z d_e$', fontsize=30)
#            axs[1].set_ylabel(r'$J(k_z)$', fontsize=30, labelpad=-1)
#            axs[1].set_title(rf't = {time}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
#            axs[1].tick_params(labelsize = 26)
#            axs[1].set_yscale('symlog')

    pos2 = axs[2].contourf(V_TE0*K_z, V_TE0*K_y, field_E_fluct_square_K)
    axs[2].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
    axs[2].set_ylabel(r'$k_y \lambda_{De0}$', fontsize=30, labelpad=-1)
    axs[2].set_title(rf'$|\delta E^2|_k$, t = {t_formatted}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
    axs[2].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos2, ax=axs[2])
    cbar.ax.tick_params(labelsize=22)

    pos3 = axs[3].contourf(V_TE0*K_z, V_TE0*K_y, JdotE_k)
    axs[3].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
    axs[3].set_ylabel(r'$k_y \lambda_{De0}$', fontsize=30, labelpad=-1)
    axs[3].set_title(rf'$(\delta J \cdot \delta E)_k$, t = {t_formatted}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
    axs[3].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos3, ax=axs[3])
    cbar.ax.tick_params(labelsize=22)
#            pos3 = axs[3].contourf(K_z, K_y, JdotE_k )
#            axs[3].set_xlabel(r'$k_z d_e$', fontsize=30)
#            axs[3].set_ylabel(r'$k_y d_e$', fontsize=30, labelpad=-1)
#            axs[3].set_title(rf'$(\delta J \cdot \delta E)_k$, t = {t_formatted}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
#            axs[3].tick_params(labelsize = 26)
#            cbar = fig.colorbar(pos3, ax=axs[3])
#            cbar.ax.tick_params(labelsize=22)

    fig.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, FILEROOT)+rf'_fft_{fignum}.png', bbox_inches='tight')
    plt.close()

In [9]:
#def main_plotting(fileroot, max_output):




if (auto_loading):

    for n in range(len(directories)):
        os.chdir(directories[n])

        input_files = []
        #loading up all input files
        for filename in os.listdir(os.getcwd()):
            if filename.endswith("~"):
                    os.remove(filename)
            elif '.lua' in filename:
                input_files.append(filename)
        #looping over all input files; selecting the fFrame and then plotting
        for i in range(len(input_files)):
            fileroot = input_files[i].split('.lua',1)[0]
            files_output_num = np.zeros(1)
            #looping over the output files to find the largest output time
            for filename in os.listdir(os.getcwd()):
                if (fileroot+'_elc') in filename:
                    output_number = re.findall(r'\d+.bp', filename)
                    if len(output_number)!=0:
                        output_number = int(output_number[0].split('.bp',1)[0])
                        files_output_num = np.append(files_output_num,output_number)
            max_output = int(np.max(files_output_num))
            if max_output==0:
                continue
            main_plotting(fileroot,max_output)
        os.chdir('..')
else:



Hello
